<a href="https://colab.research.google.com/github/ob3dd/Prediction-of-Product-Sales/blob/main/Prediction_of_Product_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prediction of Product Sales

**Obed Okoro**

##Project Overview

##Load and Inspect Data

In [33]:
import pandas as pd

fname = '/content/drive/MyDrive/CodingDojo/01-Fundamentals/Week02/Data/sales_predictions_2023.csv'

In [34]:
df_sales = pd.read_csv('/content/drive/MyDrive/CodingDojo/01-Fundamentals/Week02/Data/sales_predictions_2023.csv')

In [35]:
df_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [36]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


##Clean Data

In [37]:
# figuring out how many rows and columns

df_sales.shape

(8523, 12)

There are 8523 rows and 12 columns.

In [38]:
# the datatypes of each variable

df_sales.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [39]:
# checking if there are any duplicates.
df_sales.duplicated().sum()

0

There are no duplicates.

In [40]:
# Identifying missing values.

missing_values = df_sales.isnull().any(axis=0)


In [41]:
# Addressing the missing values by using a placeholder value. 

placeholder_value = -999
df_sales.fillna(placeholder_value, inplace=True)



In [42]:
# printing the result using the "if" function to confirm whether there are any missing values or not 

missing_values_after = df_sales.isnull().any(axis=0)

if missing_values_after.any():
    print("There are still missing values in the data.")
else:
    print("There are no missing values in the data.")


There are no missing values in the data.


In [43]:
# Finding and fixing any inconsistent categories of data.
# the "Item Fat Column" was inconsistent

item_fat_column = 'Item_Fat_Content'
item_counts = df_sales[item_fat_column].value_counts()


In [44]:
# replacing the "LF, low fat" inconsistencies with 'Low Fat' so they can be consistent.

df_sales['Item_Fat_Content'] = df_sales['Item_Fat_Content'].replace('LF''low fat', 'Low Fat')


In [45]:
# repeating the same process for the "regular"

df_sales['Item_Fat_Content'] = df_sales['Item_Fat_Content'].replace('reg', 'Regular')

In [46]:
# printing the column

print(df_sales['Item_Fat_Content'])

0       Low Fat
1       Regular
2       Low Fat
3       Regular
4       Low Fat
         ...   
8518    Low Fat
8519    Regular
8520    Low Fat
8521    Regular
8522    Low Fat
Name: Item_Fat_Content, Length: 8523, dtype: object


In [47]:
# obtaining the summary statistics of any numerical columns.

numerical_columns = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales']
summary_statistics = df_sales[numerical_columns].describe()

In [48]:
# printing ithe "summary statistics"

print(summary_statistics)

       Item_Weight  Item_Visibility     Item_MRP  Outlet_Establishment_Year  \
count  8523.000000      8523.000000  8523.000000                8523.000000   
mean   -160.830931         0.066132   140.992782                1997.831867   
std     381.595775         0.051598    62.275067                   8.371760   
min    -999.000000         0.000000    31.290000                1985.000000   
25%       6.650000         0.026989    93.826500                1987.000000   
50%      11.000000         0.053931   143.012800                1999.000000   
75%      16.000000         0.094585   185.643700                2004.000000   
max      21.350000         0.328391   266.888400                2009.000000   

       Item_Outlet_Sales  
count        8523.000000  
mean         2181.288914  
std          1706.499616  
min            33.290000  
25%           834.247400  
50%          1794.331000  
75%          3101.296400  
max         13086.964800  


##Explanatory Data Analysis